# Memento Pattern

## Intent
Capture and externalize an object's internal state without violating encapsulation, so the object can be restored to this state later.

## Problem
You need to save and restore object state but:
- Direct access to internal state violates encapsulation
- Want undo/redo functionality
- Need snapshots of object state
- Don't want to expose implementation details

**Real-world analogy**: Save game - captures game state without exposing how game works internally

## When to Use
✅ **Use when:**
- Need to save/restore object state
- Direct interface would expose implementation
- Want undo/redo mechanism
- Need checkpoints or snapshots

❌ **Avoid when:**
- State is simple and can be public
- Memory is constrained (mementos can be large)
- State changes infrequently

## Pattern Structure
```
┌───────────┐         ┌─────────┐
│Originator │────────►│ Memento │
├───────────┤         ├─────────┤
│state      │         │state    │
│save()     │         └─────────┘
│restore()  │              ▲
└───────────┘              │
                      ┌────┴─────┐
                      │Caretaker │
                      ├──────────┤
                      │mementos[]│
                      └──────────┘
```

## Example 1: Without Memento

**Problem**: Exposing internal state, violating encapsulation

In [ ]:
# WITHOUT Memento - Exposing internal state
class TextEditor:
    def __init__(self):
        self.content = ""  # Public! Bad encapsulation
        self.font = "Arial"
        self.size = 12

# Client saves state directly
editor = TextEditor()
editor.content = "Hello"

# Save state (breaks encapsulation)
saved_content = editor.content
saved_font = editor.font
saved_size = editor.size

# Make changes
editor.content = "World"

# Restore (client must know internal structure)
editor.content = saved_content
editor.font = saved_font
editor.size = saved_size

print("\n❌ Client knows internal structure!")
print("❌ Violates encapsulation!")
print("❌ Hard to add new state variables!")

## Implementation: Memento Pattern

In [ ]:
from typing import List
from datetime import datetime

# Memento: Stores state
class EditorMemento:
    """Memento storing editor state."""
    
    def __init__(self, content: str, cursor_position: int, font: str, size: int):
        self._content = content  # Private!
        self._cursor_position = cursor_position
        self._font = font
        self._size = size
        self._timestamp = datetime.now()
    
    def get_state(self) -> tuple:
        """Only originator can access state."""
        return (self._content, self._cursor_position, self._font, self._size)
    
    def get_name(self) -> str:
        """Get memento description."""
        preview = self._content[:20] + "..." if len(self._content) > 20 else self._content
        return f"{self._timestamp.strftime('%H:%M:%S')} - {preview}"


# Originator: Creates and uses mementos
class TextEditor:
    """Text editor with state."""
    
    def __init__(self):
        self._content = ""
        self._cursor_position = 0
        self._font = "Arial"
        self._size = 12
    
    def type(self, text: str) -> None:
        """Type text at cursor."""
        self._content = self._content[:self._cursor_position] + text + self._content[self._cursor_position:]
        self._cursor_position += len(text)
        print(f"  ⌨️  Typed: '{text}'")
    
    def delete(self, count: int) -> None:
        """Delete characters."""
        start = max(0, self._cursor_position - count)
        deleted = self._content[start:self._cursor_position]
        self._content = self._content[:start] + self._content[self._cursor_position:]
        self._cursor_position = start
        print(f"  ⌫  Deleted: '{deleted}'")
    
    def set_font(self, font: str, size: int) -> None:
        self._font = font
        self._size = size
        print(f"  🔤 Font changed to {font} {size}pt")
    
    def save(self) -> EditorMemento:
        """Create memento with current state."""
        print(f"  💾 Saving state...")
        return EditorMemento(self._content, self._cursor_position, self._font, self._size)
    
    def restore(self, memento: EditorMemento) -> None:
        """Restore state from memento."""
        self._content, self._cursor_position, self._font, self._size = memento.get_state()
        print(f"  ↩️  Restored to: {memento.get_name()}")
    
    def show_content(self) -> None:
        print(f"  📝 Content: '{self._content}'")


# Caretaker: Manages mementos
class History:
    """Manages editor history."""
    
    def __init__(self, editor: TextEditor):
        self._editor = editor
        self._mementos: List[EditorMemento] = []
        self._current_index = -1
    
    def save(self) -> None:
        """Save current state."""
        # Remove any states after current index (if we undid and then made changes)
        self._mementos = self._mementos[:self._current_index + 1]
        
        memento = self._editor.save()
        self._mementos.append(memento)
        self._current_index += 1
    
    def undo(self) -> None:
        """Undo to previous state."""
        if self._current_index > 0:
            self._current_index -= 1
            self._editor.restore(self._mementos[self._current_index])
        else:
            print("  ⚠️  Nothing to undo")
    
    def redo(self) -> None:
        """Redo to next state."""
        if self._current_index < len(self._mementos) - 1:
            self._current_index += 1
            self._editor.restore(self._mementos[self._current_index])
        else:
            print("  ⚠️  Nothing to redo")
    
    def show_history(self) -> None:
        print("\n  📜 History:")
        for i, memento in enumerate(self._mementos):
            marker = "→" if i == self._current_index else " "
            print(f"    {marker} {i}: {memento.get_name()}")


# Demo
print("\n=== Memento Pattern (Text Editor) ===")

editor = TextEditor()
history = History(editor)

print("\n1. Initial state:")
history.save()
editor.show_content()

print("\n2. Type 'Hello':")
editor.type("Hello")
history.save()
editor.show_content()

print("\n3. Type ' World':")
editor.type(" World")
history.save()
editor.show_content()

print("\n4. Delete 6 characters:")
editor.delete(6)
history.save()
editor.show_content()

history.show_history()

print("\n5. Undo:")
history.undo()
editor.show_content()

print("\n6. Undo again:")
history.undo()
editor.show_content()

print("\n7. Redo:")
history.redo()
editor.show_content()

history.show_history()

print("\n✅ State saved/restored without exposing internals!")

## Real-World Example: Game Save States

In [ ]:
import copy

# Memento
class GameMemento:
    """Memento storing game state."""
    
    def __init__(self, level: int, health: int, score: int, position: tuple, inventory: list):
        self._level = level
        self._health = health
        self._score = score
        self._position = position
        self._inventory = copy.deepcopy(inventory)  # Deep copy mutable state
        self._timestamp = datetime.now()
    
    def get_state(self) -> tuple:
        return (self._level, self._health, self._score, self._position, self._inventory)
    
    def get_description(self) -> str:
        return f"Level {self._level} - HP:{self._health} Score:{self._score} [{self._timestamp.strftime('%H:%M:%S')}]"


# Originator
class Game:
    """Game with state."""
    
    def __init__(self):
        self._level = 1
        self._health = 100
        self._score = 0
        self._position = (0, 0)
        self._inventory = []
    
    def play(self, action: str) -> None:
        """Simulate game actions."""
        if action == "fight":
            self._health -= 20
            self._score += 100
            print(f"  ⚔️  Fought enemy! HP: {self._health}, Score: {self._score}")
        
        elif action == "explore":
            self._position = (self._position[0] + 1, self._position[1] + 1)
            print(f"  🗺️  Explored! Position: {self._position}")
        
        elif action == "find_item":
            item = f"Item{len(self._inventory) + 1}"
            self._inventory.append(item)
            print(f"  🎁 Found {item}! Inventory: {self._inventory}")
        
        elif action == "level_up":
            self._level += 1
            self._health = 100
            print(f"  ⬆️  Level up! Now level {self._level}")
    
    def save_game(self) -> GameMemento:
        """Create save point."""
        print(f"  💾 Saving game...")
        return GameMemento(self._level, self._health, self._score, self._position, self._inventory)
    
    def load_game(self, memento: GameMemento) -> None:
        """Load save point."""
        self._level, self._health, self._score, self._position, self._inventory = memento.get_state()
        print(f"  📥 Loaded: {memento.get_description()}")
    
    def show_status(self) -> None:
        print(f"  🎮 Level {self._level} | HP: {self._health} | Score: {self._score} | Pos: {self._position} | Items: {self._inventory}")


# Caretaker
class SaveManager:
    """Manages game saves."""
    
    def __init__(self):
        self._saves: List[GameMemento] = []
    
    def save(self, memento: GameMemento, slot: int) -> None:
        if slot >= len(self._saves):
            self._saves.append(memento)
        else:
            self._saves[slot] = memento
        print(f"  💾 Saved to slot {slot}")
    
    def load(self, slot: int) -> GameMemento:
        if slot < len(self._saves):
            return self._saves[slot]
        raise IndexError("Save slot not found")
    
    def list_saves(self) -> None:
        print("\n  💾 Save Slots:")
        for i, save in enumerate(self._saves):
            print(f"    Slot {i}: {save.get_description()}")


# Demo
print("\n=== Game Save States ===")

game = Game()
save_manager = SaveManager()

print("\n1. Starting game:")
game.show_status()

print("\n2. Playing...")
game.play("explore")
game.play("find_item")
game.play("fight")
game.show_status()

print("\n3. Save checkpoint 1:")
checkpoint1 = game.save_game()
save_manager.save(checkpoint1, 0)

print("\n4. Continue playing...")
game.play("fight")
game.play("fight")
game.play("find_item")
game.show_status()

print("\n5. Save checkpoint 2:")
checkpoint2 = game.save_game()
save_manager.save(checkpoint2, 1)

print("\n6. Continue playing...")
game.play("fight")  # Oops, died!
game.show_status()

save_manager.list_saves()

print("\n7. Load checkpoint 1:")
game.load_game(save_manager.load(0))
game.show_status()

print("\n✅ Game state saved and loaded successfully!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Encapsulation**: Preserves encapsulation boundaries
2. **Undo/redo**: Easy to implement undo/redo
3. **Simplifies originator**: Doesn't manage its own history
4. **Snapshots**: Create checkpoints easily
5. **Recovery**: Can recover from errors

### ❌ Disadvantages
1. **Memory usage**: Mementos can consume lots of memory
2. **Performance**: Copying state can be expensive
3. **Lifetime management**: Who deletes old mementos?
4. **Complexity**: Additional classes to maintain

## Memento vs Command

**Memento**:
- Stores state snapshots
- Can restore entire state
- Good for complex state
- More memory intensive

**Command**:
- Stores operations
- Can undo by reversing operation
- Good for discrete operations
- More computationally intensive

## Common Use Cases

1. **Text editors**: Undo/redo
2. **Games**: Save points, checkpoints
3. **Databases**: Transactions, rollback
4. **Graphics editors**: History states
5. **Configuration**: Snapshots
6. **Wizards**: Go back to previous steps

## Related Patterns

- **Command**: Alternative undo mechanism
- **Iterator**: Traverse history of states
- **Prototype**: Memento can use cloning

## Best Practices

1. **Make memento immutable**: Prevent accidental changes
2. **Limit history size**: Don't keep unlimited mementos
3. **Use shallow copy carefully**: Be aware of reference sharing
4. **Serialize for persistence**: Save to disk if needed
5. **Compress old states**: Reduce memory usage
6. **Document what's saved**: Clear about state included
7. **Consider incremental**: Store only changes, not full state

## Python-Specific: Using copy module

Python's `copy` module simplifies memento implementation:

```python
import copy

class MyClass:
    def save(self):
        return copy.deepcopy(self)  # Create memento
    
    def restore(self, memento):
        self.__dict__ = copy.deepcopy(memento.__dict__)
```

## Summary

Memento pattern enables:
- Saving and restoring object state
- Preserving encapsulation
- Implementing undo/redo
- Creating checkpoints

Perfect for: Text editors, games, databases, graphics editors, configuration management.

**Key Insight**: Capture object state in a memento without exposing internals, enabling state restoration while preserving encapsulation!